**[中級機械学習ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


**カテゴリ変数**をエンコードすることで、これまでで最良の結果が得られる！

# セットアップ

以下の質問は作業に対するフィードバックを提供する。フィードバックシステムを設定するために次のセルを実行しよう。

In [101]:
# Kaggle学習環境のコード確認設定
# from learntools.core import binder
# binder.bind(globals())
# from learntools.ml_intermediate.ex3 import *
# print("Setup Complete")

このエクササイズでは、[Kaggle学習ユーザー向け住宅価格コンペティション](https://www.kaggle.com/c/home-data-for-ml-course)のデータを使用する。

![Ames住宅データセット画像](https://i.imgur.com/lTJVG4e.png)

次のコードセルを変更せずに実行して、訓練セットと検証セットを`X_train`、`X_valid`、`y_train`、`y_valid`に読み込む。テストセットは`X_test`に読み込まれる。

In [102]:
# --- データファイルの展開と確認 ---
#   データディレクトリ内のファイル一覧を表示する。
!ls ./input/Housing-Prices 
!echo "---------------"
# !gzip -d -c ./input/Housing-Prices/train.csv.gz > ./input/Housing-Prices/train.csv
#   train.csv.gz（圧縮ファイル）を解凍してtrain.csvを作成する。
!gzip -d -c ./input/Housing-Prices/train.csv.gz > ./input/Housing-Prices/train.csv
# !gzip -d -c ./input/Housing-Prices/test.csv.gz > ./input/Housing-Prices/test.csv
#   test.csv.gz（圧縮ファイル）を解凍してtest.csvを作成する。
!gzip -d -c ./input/Housing-Prices/test.csv.gz > ./input/Housing-Prices/test.csv
# !ls ./input/Housing-Prices
#   再度ファイル一覧を表示し、解凍後のファイルが存在するか確認する。
!ls ./input/Housing-Prices 


88875.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


test.csv  test.csv.gz  train.csv  train.csv.gz


88880.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


---------------


88885.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
88890.91s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
88896.05s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


test.csv  test.csv.gz  train.csv  train.csv.gz


In [103]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読み込み
# index_col='Id'でIDを行インデックスとして設定
X = pd.read_csv('./input/Housing-Prices/train.csv', index_col='Id') 
X_test = pd.read_csv('./input/Housing-Prices/test.csv', index_col='Id')

# 目的変数が欠損している行を削除し、目的変数と説明変数を分離
# dropna: 欠損値を含む行を削除
# axis=0: 行方向の削除
# subset=['SalePrice']: SalePriceが欠損している行のみを対象
# inplace=True: 元のデータフレームを変更
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
# 目的変数（住宅価格）を取り出す
y = X.SalePrice
# 説明変数から目的変数の列を削除
X.drop(['SalePrice'], axis=1, inplace=True)

# シンプルにするため、欠損値を含む列を削除
# リスト内包表記を使用: X.columns内の各列colに対して、X[col].isnull().any()がTrueなら列名を取得
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# 訓練データから検証データを分割
# train_size=0.8: 訓練データに80%を使用
# test_size=0.2: 検証データに20%を使用
# random_state=0: 再現性のための乱数シード
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

次のコードセルを使用して、データの最初の5行を表示する。

In [104]:
# head()メソッドでデータフレームの先頭5行を表示
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


このデータセットには数値変数とカテゴリ変数の両方が含まれていることに注目。モデルを訓練する前にカテゴリデータをエンコードする必要がある。

異なるモデルを比較するために、チュートリアルと同じ`score_dataset()`関数を使用する。この関数はランダムフォレストモデルから[平均絶対誤差](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE)を報告する。

In [105]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 異なるアプローチを比較するための関数
def score_dataset(X_train, X_valid, y_train, y_valid):
    # ランダムフォレスト回帰モデルを初期化（100本の決定木、乱数シード0）
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    # モデルを訓練データにフィット（学習）させる
    model.fit(X_train, y_train)
    # 検証データに対する予測を生成
    preds = model.predict(X_valid)
    # 予測値と実際の値の平均絶対誤差を計算して返す
    return mean_absolute_error(y_valid, preds)

# ステップ1: カテゴリデータを含む列を削除する

最も単純なアプローチから始める。以下のコードセルを使用して、`X_train`と`X_valid`のデータを前処理し、カテゴリデータを含む列を削除する。前処理されたDataFrameをそれぞれ`drop_X_train`と`drop_X_valid`に設定する。

In [106]:
# 訓練データと検証データからカテゴリ列を削除
# select_dtypes(exclude=['object'])で文字列型（オブジェクト型）の列を除外
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

# Kaggle環境での回答確認用コード（コメントアウト）
# step_1.check()

In [107]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_1.hint()
#step_1.solution()

このアプローチのMAEを取得するために次のコードセルを実行する。

In [108]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


# ステップ2: ラベルエンコーディング

## ラベルエンコーディングとは何か？

ラベルエンコーディングは、カテゴリ変数（文字列）を数値に変換する手法である。
例えば、['赤', '青', '緑'] というカテゴリを [0, 1, 2] という数値に変換する。

なぜラベルエンコーディングが必要なのか？
1. 多くの機械学習アルゴリズムは数値データしか扱えないため
2. 文字列のままでは計算ができないため
3. 特に決定木ベースのアルゴリズム（ランダムフォレストなど）では、
   カテゴリ変数を数値に変換することで効率的に学習できる

ラベルエンコーディングに進む前に、データセットを調査する。具体的には、`'Condition2'`列を見てみる。以下のコードセルは、訓練セットと検証セットの両方でのユニークな値を表示する。

In [109]:
# unique()メソッドで列内のユニークな値を取得
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


ここで、以下のコードを書いた場合：

- 訓練データにラベルエンコーダーをフィットし、
- それを使って訓練データと検証データの両方を変換する

エラーが発生する。なぜこれが問題になるか分かるだろうか？（この質問に答えるには、上記の出力を使用する必要がある）

上記の出力を見ると、訓練データと検証データで異なる値が含まれていることがわかる：

- 訓練データには 'PosA', 'RRAe' が含まれている
- 検証データには 'RRAn', 'RRNn' が含まれている

これが問題になる理由：

1. ラベルエンコーダーは訓練データに存在する値だけを学習する
2. 検証データに訓練データにない値（'RRAn', 'RRNn'）が含まれていると、
   エンコーダーはこれらの値に対応する数値を持っていないためエラーになる
3. scikit-learn の LabelEncoder は未知のカテゴリに対して例外を発生させる設計になっている


In [110]:
# Kaggle環境でのヒント表示用（コメントアウト）
# step_2.a.hint()

In [111]:
# Kaggle環境での解答表示用（コメントアウト）
# step_2.a.solution()

これは実世界のデータでよく遭遇する一般的な問題であり、この問題を解決するためのアプローチは多数ある。例えば、新しいカテゴリに対応するカスタムラベルエンコーダーを作成することができる。しかし、最も単純なアプローチは、問題のあるカテゴリ列を削除することだ。

この問題に対する可能な解決策：
1. カスタムエンコーダーを作成し、未知の値に特別な数値（例：-1）を割り当てる
2. 訓練データと検証/テストデータを結合してからエンコーディングする（データリーク注意）
3. 問題のある列を削除する（情報損失のトレードオフ）

ここでは単純さを優先して3番目のアプローチを採用する。なぜなら：
- 実装が簡単で確実
- 未知のカテゴリが将来も出現する可能性がある場合、根本的な解決になる
- 多くの列がある場合、一部を削除しても全体のパフォーマンスへの影響は限定的

以下のコードセルを実行して、問題のある列をPythonリスト`bad_label_cols`に保存する。同様に、安全にラベルエンコードできる列は`good_label_cols`に格納される。

In [112]:
# すべてのカテゴリ列
# dtype == "object"でカテゴリ（文字列）型の列を特定
# Pythonのリスト内包表記を使用して、データ型が「object」（文字列）の列だけを抽出している
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# 安全にラベルエンコードできる列
# 訓練データと検証データで同じ値のセットを持つ列
# なぜこの処理が必要か？
# 1. set(X_train[col])：訓練データの特定の列に含まれるユニークな値の集合を取得
# 2. set(X_valid[col])：検証データの同じ列に含まれるユニークな値の集合を取得
# 3. 両者が等しい（==）場合、その列は安全にラベルエンコードできる
#
# 【重要な注意点】
# 実際には、訓練データ ⊇ 検証データ（訓練データが検証データを包含）の関係であれば
# 安全にラベルエンコードできる。なぜなら：
# - 訓練データに存在する値だけが検証データにある場合、すべての値に対応する数値が存在する
# - 訓練データに{'赤','青','緑','黄'}、検証データに{'赤','青'}のような場合も安全
#
# しかし、このコードでは完全一致（==）を条件としているため、より厳格な条件になっている。
# より正確には以下のようなコードが適切：
# good_label_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]
#
# 現在のコードは簡潔さを優先しているが、訓練データに余分な値がある場合も
# 安全にエンコードできる可能性を排除している点に注意。
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# データセットから削除される問題のある列
# 集合の差集合演算で、すべてのカテゴリ列から安全な列を引く
# なぜこの処理をするのか？
# 1. set(object_cols)：すべてのカテゴリ列の集合
# 2. set(good_label_cols)：安全にエンコードできる列の集合
# 3. 差集合演算（-）：1から2を引くことで、安全でない列だけを抽出
# 4. list()で結果をリストに変換
# 例：全カテゴリ列が{'色','形','サイズ','ブランド'}で、安全な列が{'色','形'}なら
#    問題のある列は{'サイズ','ブランド'}となる
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Utilities', 'ExterCond', 'Foundation', 'SaleType', 'Condition2', 'LandSlope', 'RoofMatl', 'Exterior2nd', 'HeatingQC', 'Neighborhood', 'Heating', 'Functional', 'Exterior1st', 'RoofStyle', 'Condition1']


以下のコードセルを使用して、`X_train`と`X_valid`のデータをラベルエンコードする。前処理されたDataFrameをそれぞれ`label_X_train`と`label_X_valid`に設定する。

ラベルエンコーディングの実装手順：
1. 問題のある列（bad_label_cols）を削除する
2. 安全な列（good_label_cols）に対してLabelEncoderを適用する
3. 訓練データでエンコーダーをfit_transformし、検証データではtransformのみを行う

なぜfit_transformとtransformを分けるのか？
- fit_transform: エンコーダーがカテゴリと数値のマッピングを学習し、変換も行う
- transform: 既に学習したマッピングを使って変換のみを行う

これにより、訓練データと検証データで一貫したエンコーディングが保証される。
例えば、訓練データで「赤」→0、「青」→1、「緑」→2と学習したら、
検証データでも同じマッピングを適用する必要がある。

- `bad_label_cols`のカテゴリ列をデータセットから削除するコードを提供している。
- `good_label_cols`のカテゴリ列をラベルエンコードする必要がある。

In [113]:
from sklearn.preprocessing import LabelEncoder

# エンコードされないカテゴリ列を削除
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# ラベルエンコーダーを適用
labelEncoder = LabelEncoder()
for col in good_label_cols:
    # 訓練データでエンコーダーをフィットし、変換
    label_X_train[col] = labelEncoder.fit_transform(X_train[col])
    # 検証データを変換（フィットはしない）
    label_X_valid[col] = labelEncoder.transform(X_valid[col])
    
# Kaggle環境での回答確認用（コメントアウト）
# step_2.b.check()

In [114]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_2.b.hint()
#step_2.b.solution()

このアプローチのMAEを取得するために次のコードセルを実行する。

ラベルエンコーディングの効果を評価する理由：
1. カテゴリ変数を削除するアプローチ（ステップ1）と比較して、
   情報を保持しながら数値化することでモデルの性能が向上するか確認する
2. カテゴリ情報をモデルに取り込むことの重要性を定量的に評価する
3. 後のステップで試すワンホットエンコーディングとの比較ベースラインを作る

In [115]:
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding):


17575.291883561644


# ステップ3: カーディナリティの調査

これまで、カテゴリ変数を扱うための2つの異なるアプローチを試した。そして、カテゴリデータをエンコードすることが、データセットから列を削除するよりも良い結果をもたらすことがわかった。

なぜカテゴリデータのエンコードが列の削除より良いのか？
1. 情報保持：カテゴリ変数には予測に役立つ情報が含まれている
   - 例えば「Street」列の「Pave」か「Grvl」かという情報は住宅価格に影響する
2. パターン認識：機械学習モデルはカテゴリ情報から重要なパターンを学習できる
   - 特に決定木ベースのモデル（ランダムフォレスト）はカテゴリ変数を効果的に扱える
3. 実証的結果：MAEの比較から、カテゴリ情報を保持する方が予測精度が向上することが確認できた
   - ステップ1（削除）のMAE: 17837.83 vs ステップ2（ラベルエンコード）のMAE: 17575.29

次に、ワンホットエンコーディングを試す。その前に、もう一つ追加のトピックをカバーする必要がある。まず、次のコードセルを変更せずに実行する。

In [116]:
# カテゴリデータを持つ各列のユニークな値の数を取得
# map関数で各列に対してnunique()を適用
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
# 列名とユニーク値数の辞書を作成
d = dict(zip(object_cols, object_nunique))

# 列ごとのユニークな値の数を昇順で表示
# sorted関数でキーの値（ユニーク値の数）に基づいてソート
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

上記の出力は、カテゴリデータを持つ各列について、その列のユニークな値の数を示している。例えば、訓練データの`'Street'`列には、砂利道を表す`'Grvl'`と舗装道路を表す`'Pave'`という2つのユニークな値がある。

カテゴリ変数のユニークな値の数を、そのカテゴリ変数の**カーディナリティ**と呼ぶ。例えば、`'Street'`変数のカーディナリティは2である。

カーディナリティとは何か？
- カテゴリ変数のユニークな値の数を「カーディナリティ」と呼ぶ
- 例：'Street'列のカーディナリティは2（'Grvl'と'Pave'の2種類）
- 'Neighborhood'列のカーディナリティは25（25種類の異なる地域名）

なぜカーディナリティが重要なのか？
1. エンコーディング方法の選択に影響する
   - カーディナリティが低い→ワンホットエンコーディングが適切
   - カーディナリティが高い→ラベルエンコーディングが効率的
2. モデルの複雑さと学習効率に影響する
   - カーディナリティが高いとモデルの複雑さが増し、過学習リスクも高まる
3. データセットのサイズと計算コストに直接関わる
   - 次のセクションで詳しく説明するように、ワンホットエンコーディングはデータサイズを大幅に増加させる

上記の出力を使用して、以下の質問に答える。

In [117]:
# 訓練データ内でカーディナリティが10より大きいカテゴリ変数はいくつありますか？
high_cardinality_numcols = 3

# 訓練データ内の'Neighborhood'変数をワンホットエンコードするには何列必要ですか？
num_cols_neighborhood = 25

# Kaggle環境での回答確認用（コメントアウト）
# step_3.a.check()

In [118]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_3.a.hint()
#step_3.a.solution()

多くの行を持つ大規模なデータセットでは、ワンホットエンコーディングによってデータセットのサイズが大幅に拡大する可能性がある。このため、通常はカーディナリティが比較的低い列のみをワンホットエンコードする。そして、カーディナリティが高い列はデータセットから削除するか、ラベルエンコーディングを使用する。

ワンホットエンコーディングとカーディナリティの関係

なぜカーディナリティが高い変数にワンホットエンコーディングを避けるのか？
1. 次元の爆発：カーディナリティが高い変数をワンホットエンコードすると、
   元の1列がカテゴリの数だけの列に展開される
2. メモリ効率：高カーディナリティ変数のワンホットエンコードは大量のメモリを消費する
3. 計算コスト：列数の増加により、モデルの訓練時間が大幅に増加する
4. 過学習リスク：特徴量が増えすぎると、モデルが訓練データに過剰適合する可能性が高まる

実際の選択基準：
- カーディナリティが低い（通常10未満）：ワンホットエンコーディングが適切
- カーディナリティが高い：ラベルエンコーディングか、重要でなければ削除を検討

例として、10,000行のデータセットで、100のユニークなエントリを持つ1つのカテゴリ列を考える。
- この列を対応するワンホットエンコーディングに置き換えると、データセットにいくつのエントリが追加されるか？
- 代わりに列をラベルエンコーディングに置き換えると、いくつのエントリが追加されるか？

回答を使用して、以下の行を埋める。

In [119]:
# 列をワンホットエンコーディングに置き換えることでデータセットに追加されるエントリ数は？
# 10,000行 × (100-1)列 = 990,000エントリ
# 元の列を削除して100列追加するので、実質99列の増加
# 
# ワンホットエンコーディングの計算方法：
# - 元の列：10,000行 × 1列 = 10,000エントリ
# - ワンホットエンコード後：10,000行 × 100列 = 1,000,000エントリ
# - 純増加：1,000,000 - 10,000 = 990,000エントリ
OH_entries_added = 10000 * 99

# 列をラベルエンコーディングに置き換えることでデータセットに追加されるエントリ数は？
# ラベルエンコーディングは元の列を整数に置き換えるだけなので、追加エントリはない
# 
# ラベルエンコーディングの計算方法：
# - 元の列：10,000行 × 1列 = 10,000エントリ
# - ラベルエンコード後：10,000行 × 1列 = 10,000エントリ（変わらない）
# - 純増加：0エントリ（元の列を置き換えるだけなので増加なし）
# 
# この違いが重要な理由：
# - データセットのサイズはメモリ使用量に直結する
# - 特徴量の数はモデルの複雑さと訓練時間に影響する
# - 実務では、この差が処理可能かどうかの分かれ目になることがある
label_entries_added = 0

# Kaggle環境での回答確認用（コメントアウト）
# step_3.b.check()

In [120]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_3.b.hint()
#step_3.b.solution()

# ステップ4: ワンホットエンコーディング

## ワンホットエンコーディングとは何か？

ワンホットエンコーディングは、カテゴリ変数を複数の二値特徴量（0か1のみの値を持つ列）に変換する手法である。各カテゴリに対して新しい列を作成し、該当するカテゴリの場合は1、それ以外は0を設定する。

例：「色」という列に「赤」「青」「緑」という値がある場合
- 「色_赤」「色_青」「色_緑」という3つの列に変換
- 「赤」のデータは [1, 0, 0]
- 「青」のデータは [0, 1, 0]
- 「緑」のデータは [0, 0, 1] となる

## なぜワンホットエンコーディングが必要なのか？

1. **順序関係の排除**: ラベルエンコーディングでは「赤=0, 青=1, 緑=2」のように数値に変換するが、これは「緑>青>赤」という順序関係を暗黙的に導入してしまう。多くのカテゴリでは、このような順序関係は存在しない。ワンホットエンコーディングはこの問題を解決する。

2. **線形モデルとの相性**: 線形回帰やロジスティック回帰などの線形モデルは、特徴量の線形結合を学習する。ラベルエンコードされた変数では、カテゴリ間の数値的な差が意味を持ってしまうが、ワンホットエンコーディングではそれぞれのカテゴリが独立した特徴量として扱われる。

   なぜこれが重要なのか？線形モデルの数学的な観点から説明すると：
   
   - 線形モデルは基本的に `y = w₁x₁ + w₂x₂ + ... + wₙxₙ + b` という形式で予測を行う
   - ラベルエンコーディングの場合：例えば「色」が「赤=0, 青=1, 緑=2」とエンコードされると、モデルは「緑」を「赤」の2倍、「青」の2倍として扱う。つまり `w_色 × 2` という重みづけになる
   - しかし実際には「緑」は「赤」の2倍ではなく、単に別のカテゴリに過ぎない
   - ワンホットエンコーディングでは：
     * 「赤」の場合：`w_赤 × 1 + w_青 × 0 + w_緑 × 0`
     * 「青」の場合：`w_赤 × 0 + w_青 × 1 + w_緑 × 0`
     * 「緑」の場合：`w_赤 × 0 + w_青 × 0 + w_緑 × 1`
   - これにより、各カテゴリは独自の重み（w_赤, w_青, w_緑）を持ち、互いに独立して影響を与えることができる

   具体例：住宅価格予測において「地域」というカテゴリ変数がある場合
   - ラベルエンコード：「都心=0, 郊外=1, 田舎=2」とすると、モデルは「田舎」を「都心」の2倍として扱ってしまう
   - 実際には「都心」「郊外」「田舎」は単に異なる地域であり、数値的な大小関係はない
   - ワンホットエンコードでは、各地域が独立した特徴量となり、それぞれが住宅価格に与える影響を個別に学習できる

3. **決定木との違い**: 決定木ベースのモデル（ランダムフォレストなど）はラベルエンコーディングでも問題なく機能するが、線形モデルではワンホットエンコーディングが必要なことが多い。

   なぜ決定木モデルはラベルエンコーディングでも問題ないのか？
   
   - 決定木の仕組み：決定木は「特徴量Xの値がしきい値Tより大きいか小さいか」という二分岐の連続で予測を行う
   - 例えば「色」が「赤=0, 青=1, 緑=2」とラベルエンコードされている場合：
     * 決定木は「色 <= 0.5」という分岐で「赤」とそれ以外を分けられる
     * 次に「色 <= 1.5」という分岐で「青」と「緑」を分けられる
   - つまり、決定木は複数の分岐を使って、ラベルエンコードされた値を効果的に「カテゴリ」として扱える
   - 決定木は特徴量の「順序」ではなく「値によるデータの分割」に基づいて学習するため、ラベルエンコードでも問題ない

   ランダムフォレストなどのアンサンブル手法も、基本的に決定木の集合であるため、同様の理由でラベルエンコーディングで効果的に機能する。

   対照的に、線形モデルは特徴量の「値」と「重み」の積の和で予測するため、カテゴリ変数の数値表現に敏感であり、ワンホットエンコーディングが必要になる。

## ワンホットエンコーディングの欠点

1. **次元の増加**: カテゴリの数だけ新しい列が増えるため、データの次元が大幅に増加する。

2. **スパース性**: 多くの0と少数の1からなる「スパース」なデータになり、計算効率が低下する場合がある。

3. **メモリ消費**: カーディナリティが高い変数では、非常に多くのメモリを消費する。

このステップでは、ワンホットエンコーディングを試す。ただし、データセット内のすべてのカテゴリ変数をエンコードするのではなく、カーディナリティが10未満の列に対してのみワンホットエンコーディングを作成する。これは上記の欠点を考慮した実用的なアプローチである。

以下のコードセルを変更せずに実行して、ワンホットエンコードされる列を含むPythonリスト`low_cardinality_cols`を設定する。同様に、`high_cardinality_cols`にはデータセットから削除されるカテゴリ列のリストが含まれる。

In [121]:
# ワンホットエンコードされる列
# カーディナリティが10未満の列を選択
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# データセットから削除される列
# 集合の差集合演算で、すべてのカテゴリ列から低カーディナリティ列を引く
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood', 'Exterior1st', 'Exterior2nd']


次のコードセルを使用して、`X_train`と`X_valid`のデータをワンホットエンコードする。前処理されたDataFrameをそれぞれ`OH_X_train`と`OH_X_valid`に設定する。

- データセット内のカテゴリ列の完全なリストはPythonリスト`object_cols`にある。
- `low_cardinality_cols`のカテゴリ列のみをワンホットエンコードする必要がある。他のすべてのカテゴリ列はデータセットから削除する必要がある。

In [99]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoderを初期化
# handle_unknown='ignore': 未知のカテゴリに対してエラーを発生させない
# sparse_output=False: 密な配列を返す（デフォルトはスパース行列）
# 注意: scikit-learnの新しいバージョンでは、sparse=Falseではなくsparse_output=Falseを使用する
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 低カーディナリティ列をワンホットエンコード
# なぜfit_transformとtransformを分けるのか？
# 1. fit_transform: エンコーダーがカテゴリと数値のマッピングを学習し、変換も行う
#    - 訓練データでのみ使用し、カテゴリの種類と順序を記憶する
# 2. transform: 既に学習したマッピングを使って変換のみを行う
#    - 検証データや未知データに適用する際に使用
#    - 訓練データと同じマッピングを保証するため重要
#
# この一貫性がなぜ重要か？
# - 機械学習モデルは特徴量の位置に基づいて学習するため、
#   訓練と予測で特徴量の順序や意味が変わると正しく予測できない
# - 例：訓練データで「色_赤」が3列目、「色_青」が4列目なら、
#   検証データでも同じ位置関係を維持する必要がある
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# ワンホットエンコーディングによりインデックスが削除されたので、元に戻す
# なぜインデックスを戻す必要があるのか？
# 1. OneHotEncoderはnumpy配列を返すため、元のDataFrameのインデックス情報が失われる
# 2. 後でconcatする際に、インデックスが一致していないとデータが正しく結合されない
# 3. 元のインデックスには行の識別情報（例：住宅ID）が含まれている可能性がある
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# カテゴリ列を削除（ワンホットエンコーディングに置き換える）
# なぜすべてのカテゴリ列（object_cols）を削除するのか？
# 1. 低カーディナリティ列はワンホットエンコード済みで別途追加するため
# 2. 高カーディナリティ列は前述の理由（メモリ消費、次元の爆発）で除外するため
# 3. 元のカテゴリ列をそのまま残すと、同じ情報が重複して含まれることになる
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# 数値特徴量にワンホットエンコードされた列を追加
# なぜconcatを使うのか？
# 1. 数値特徴量とワンホットエンコードされた特徴量を横方向（列方向）に結合するため
# 2. axis=1は列方向の結合を指定（axis=0は行方向の結合）
# 3. これにより、元の数値特徴量とカテゴリ特徴量の両方の情報を保持したデータセットが作成される
#
# この結合後のデータセットの特徴：
# - すべての数値特徴量が含まれる
# - カーディナリティが低いカテゴリ変数がワンホットエンコードされて含まれる
# - カーディナリティが高いカテゴリ変数は除外されている
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# 列名をすべて文字列に変換
# なぜ必要か？
# 1. OneHotEncoderは数値の列名を生成し、pd.DataFrameに変換すると列名に整数型が使われる
# 2. 元のnum_X_trainの列名は文字列型
# 3. pd.concatで結合すると、列名に整数型と文字列型が混在する
# 4. scikit-learnは列名の型が混在していると「TypeError: Feature names are only supported if all input features have string names」エラーを出す
# 5. 列名をすべて文字列型に統一することでエラーを解消できる
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

# Kaggle環境での回答確認用（コメントアウト）
# step_4.check()

In [68]:
# ヒントや解答コードを表示するための行（Kaggle環境用）
#step_4.hint()
#step_4.solution()

このアプローチのMAEを取得するために次のコードセルを実行する。

## ワンホットエンコーディングの効果を評価する

ここでは、ワンホットエンコーディングを使用したアプローチの性能を評価する。これにより、以下の点が明らかになる：

1. **カテゴリ変数の表現方法による影響**：
   - ラベルエンコーディング（順序関係を暗黙的に導入）
   - ワンホットエンコーディング（各カテゴリを独立した特徴として表現）
   のどちらが予測精度に良い影響を与えるか

2. **カーディナリティによる選択的処理の効果**：
   - 低カーディナリティ列のみをワンホットエンコード
   - 高カーディナリティ列を削除
   という選択が妥当かどうか

3. **モデルとエンコーディング手法の相性**：
   - ランダムフォレスト（決定木ベース）はラベルエンコーディングでも効果的に機能するが、
     ワンホットエンコーディングでさらに性能が向上するか
   - 線形モデルであれば、ワンホットエンコーディングの効果はより顕著になる可能性がある

結果を見ることで、どのエンコーディング手法が最も効果的かを判断できる。

In [100]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084


# ステップ5: テスト予測を生成し、結果を提出する

ステップ4を完了した後、学んだことを使用してリーダーボードに結果を提出したい場合は、予測を生成する前にテストデータを前処理する必要がある。

**このステップは完全にオプションであり、エクササイズを正常に完了するためにリーダーボードに結果を提出する必要はない。**

[コンペティションに参加する](https://www.kaggle.com/c/home-data-for-ml-course)方法や結果をCSVに保存する方法を思い出すのに助けが必要な場合は、前のエクササイズを確認してください。結果ファイルを生成したら、以下の手順に従ってください：
- 右上隅の青い**COMMIT**ボタンをクリックする。これによりポップアップウィンドウが生成される。
- コードの実行が終了したら、ポップアップウィンドウの右上にある青い**Open Version**ボタンをクリックする。これにより、同じページのビューモードに移動する。これらの指示に戻るには下にスクロールする必要がある。
- 画面左側の**Output**タブをクリックする。次に、**Submit to Competition**ボタンをクリックして、結果をリーダーボードに提出する。
- パフォーマンスを向上させるためにさらに作業を続けたい場合は、画面右上の青い**Edit**ボタンを選択する。その後、モデルを変更してプロセスを繰り返すことができる。

In [ ]:
# テストデータの欠損値を0で埋める
# なぜ欠損値を0で埋めるのか？
# 1. 一貫性の確保：訓練データと同じ前処理をテストデータにも適用する必要がある
#    - 訓練データでも欠損値を0で埋めていた場合、テストデータも同様に処理する
# 2. モデルの期待：モデルは訓練時に見た形式のデータを予測時にも期待する
#    - 訓練時と異なる形式のデータを与えると、予測精度が低下する可能性がある
# 3. 実用的な選択：単純な方法だが、多くの場合で十分に機能する
#
# 代替手法：
# - 平均値や中央値での補完：数値的に意味のある値で置き換える
# - 高度な補完手法：KNN、回帰モデル、多重代入法などを使用
# - 特別な値の使用：欠損を示す特別な値（例：-999）を使用し、別途「欠損フラグ」列を追加
X_test.fillna(0, inplace=True)

# すべてのカテゴリ列
# なぜX_testではなくXを使うのか？
# 1. 一貫性の確保：訓練データ（X）で識別したカテゴリ列と同じ列をテストデータでも処理する
# 2. 完全性：テストデータには訓練データにあるカテゴリが存在しない可能性がある
# 3. モデルの期待：モデルは訓練データの構造に基づいて学習しているため、
#    テストデータも同じ構造にする必要がある
object_cols = [col for col in X.columns if X[col].dtype == "object"]

# 安全にラベルエンコードできる列
# テストデータの値が訓練データの値のサブセットであることを確認
# なぜこのチェックが重要なのか？
# 1. エンコーディングの安全性：ラベルエンコーダーは訓練データで見たカテゴリのみを変換できる
# 2. エラー回避：テストデータに新しいカテゴリがあると、変換時にエラーが発生する
# 3. 予測の信頼性：訓練データに存在しないカテゴリは、モデルが学習していないため
#    適切に予測できない可能性が高い
#
# issubset()の意味：
# - set(X_test[col]).issubset(set(X[col])) は「テストデータの値がすべて訓練データに含まれているか」を確認
# - これにより、安全にエンコードできる列だけを選択できる
good_label_cols = [col for col in object_cols if 
                   set(X_test[col]).issubset(set(X[col]))]
        
# データセットから削除される問題のある列
# なぜ問題のある列を削除するのか？
# 1. エラー回避：新しいカテゴリ値があるとエンコーディング時にエラーが発生する
# 2. 予測の安定性：訓練データに存在しないカテゴリを含む列は予測に悪影響を与える可能性がある
# 3. 実用的な選択：複雑なカスタムエンコーダーを作成するよりも、単純に削除する方が実装が容易
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

# エンコードされないカテゴリ列を削除
# なぜ訓練データ全体（X）を使うのか？
# 1. 最終モデルの構築：これまでの分析では訓練データと検証データを分けていたが、
#    最終モデルでは全データを使用するのが一般的
# 2. データ量の最大化：より多くのデータでモデルを訓練することで、一般化性能が向上する
# 3. 実世界での予測：実際のコンペティションや本番環境では、利用可能なすべてのデータを
#    使ってモデルを訓練するのが標準的
label_X = X.drop(bad_label_cols, axis=1)
label_X_test = X_test.drop(bad_label_cols, axis=1)

# ラベルエンコーダーを適用
# なぜ各列に対して新しいエンコーダーインスタンスを使うのか？
# 1. 列ごとの独立性：各カテゴリ列は独自のカテゴリセットを持つため、個別にエンコードする必要がある
# 2. マッピングの一貫性：各列内でのカテゴリ→数値のマッピングを一貫させるため
# 3. エンコーダーの仕様：LabelEncoderは単一の特徴（列）に対してのみ動作するよう設計されている
labelEncoder = LabelEncoder()
for col in good_label_cols:
    # 全訓練データでエンコーダーをフィットし、変換
    # なぜfit_transformとtransformを分けるのか？
    # 1. 学習と適用の分離：訓練データでカテゴリ→数値のマッピングを学習し、
    #    そのマッピングをテストデータに適用する
    # 2. データリーク防止：テストデータの情報を使ってエンコーダーを学習させると、
    #    未来の情報を使って過去を予測することになり、不適切
    # 3. 一貫性の確保：同じカテゴリに対して常に同じ数値を割り当てる必要がある
    label_X[col] = labelEncoder.fit_transform(X[col])
    # テストデータを変換（フィットはしない）
    label_X_test[col] = labelEncoder.transform(X_test[col])

# 200の決定木を持つモデル、MAE: 15923.57616
#
# このノートブックの目的：住宅価格の予測
# - 様々な特徴（カテゴリ変数を含む）から住宅の販売価格（SalePrice）を予測する
# - これまでのステップでは、カテゴリ変数の処理方法（削除、ラベルエンコード、ワンホットエンコード）を比較した
# - ステップ5では、最適な方法を使って最終的なモデルを構築し、テストデータに対する予測を生成する
#
# ランダムフォレストを使う理由：
# - 複雑な非線形関係を捉えられる（住宅特性と価格の関係は単純な線形関係ではない）
# - カテゴリ変数と数値変数の両方を効果的に扱える
# - 外れ値に対して堅牢（不動産市場には極端に高価な物件などが存在する可能性がある）
# - 特徴量の重要度を評価できる（どの特徴が価格に最も影響するかを分析できる）
#
# n_estimators=200（決定木の数）を選んだ理由：
# 1. 予測精度の向上：決定木を増やすと、個々の木の誤差が平均化され、より正確な価格予測が可能になる
#    - 前のステップでは100本の決定木を使用していたが、最終モデルでは200本に増やして精度を向上
# 2. 過学習の防止：複数の木を使うことで、訓練データに過剰適合するリスクを減らし、
#    未知の物件に対しても信頼性の高い価格予測ができる
# 3. 経験則：住宅価格予測のような複雑な問題では、200本程度の決定木が良いバランスを提供する
#
# 注意点：
# - 決定木の数を増やすと計算コストも増加する（大規模なデータセットでは考慮が必要）
# - ある程度以上増やしても予測精度の向上が頭打ちになる場合が多い
# - 理想的には、クロスバリデーションを使って最適な木の数を決定する
model = RandomForestRegressor(n_estimators=200, random_state=0)
model.fit(label_X, y)

# テスト予測を取得
# なぜmodel.predictを使うのか？
# 1. 学習済みモデルの活用：fit()で学習したモデルのパターンを使って新しいデータを予測する
# 2. 推論フェーズ：訓練（fit）と予測（predict）は機械学習の2つの主要フェーズ
# 3. 回帰問題：住宅価格予測は回帰問題なので、predict()は各サンプルに対する連続値（価格）を返す
#
# 予測値の解釈：
# - 各行（住宅）に対する予測価格が返される
# - 予測値の単位は目的変数（y）と同じ（この場合はドル）
# - 予測値は小数点を含む連続値として返される
preds_test = model.predict(label_X_test)

# テスト予測をファイルに保存
# なぜこの形式で保存するのか？
# 1. コンペティション要件：Kaggleコンペティションでは特定の形式での提出が求められる
#    - 通常、'Id'列と予測対象の列（ここでは'SalePrice'）が必要
# 2. インデックスの活用：label_X_test.indexを使うことで、元のデータの識別子を保持できる
# 3. フォーマット要件：index=Falseを指定することで、DataFrameのインデックスを別列として
#    出力せず、Kaggleの要求する形式に合わせている
#
# CSVフォーマットを選ぶ理由：
# - 互換性：ほとんどのシステムで読み込み可能な標準フォーマット
# - 人間可読性：テキストエディタで開いて内容を確認できる
# - Kaggle標準：多くのKaggleコンペティションでCSV形式での提出が求められる
output = pd.DataFrame({'Id': label_X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

# 次のステップ

欠損値の処理とカテゴリエンコーディングにより、モデリングプロセスは複雑になっている。この複雑さは、将来使用するためにモデルを保存したい場合にさらに悪化する。この複雑さを管理するための鍵は**パイプライン**と呼ばれるものだ。

**[パイプラインの使用方法を学ぶ](https://www.kaggle.com/alexisbcook/pipelines)**で、カテゴリ変数、欠損値、およびデータが投げかけるその他の厄介な問題を含むデータセットを前処理する方法を学ぼう。

---
**[中級機械学習ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**





*質問やコメントがありますか？[Learn Discussion forum](https://www.kaggle.com/learn-forum)で他の学習者とチャットしましょう。*